# Global Shark Attack Incidents Data Analysis

Using the data set in <https://www.kaggle.com/teajay/global-shark-attacks/> (Version 7)

After cleaning and analyzing the data, we want to answer the following questions:

* Attacks per year 1900+ (Total, Fatal, Non Fatal) ?


* Relation of ocean temperature and shark attack?


## Import and cleaning data set

### Import libraries and data set's

In [60]:
import pandas as pd
import numpy as np

In [61]:
#read dataframe and create a backup
df = pd.read_csv('attacks.csv', sep = ',', encoding='latin-1')
df_backup = df.copy()

### Clean data frame

#### work with column names

In [62]:
columns_name = df.columns
# make remove spaces before and after and put all in lower case
columns_name = [item.strip().lower() for item in columns_name]
# replace spaces for underline
columns_name = [item.replace(' ', '_') for item in columns_name]
df.columns = columns_name

#### Work with duplicate row's /columns with missing values

In [63]:
df.head()

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [64]:
#drop duplicate rows
df.drop_duplicates(subset =df.columns, inplace = True)
df.shape

(6312, 24)

In [66]:
#count NaN in each row
df['count_missing'] = df.isnull().sum(axis=1)
#using a mask to eliminate rows with 20 missing values or more
mask = df['count_missing'] < 20
df = df.loc[mask,:]
#drop column use before
df.drop(labels='count_missing', axis=1, inplace=True)

In [67]:
df.shape

(6302, 24)

In [68]:
#Count missing values in Columns
df.isnull().sum()

case_number                  1
date                         0
year                         2
type                         4
country                     50
area                       455
location                   540
activity                   544
name                       210
sex                        565
age                       2831
injury                      28
fatal_(y/n)                539
time                      3354
species                   2838
investigator_or_source      17
pdf                          0
href_formula                 1
href                         0
case_number.1                0
case_number.2                0
original_order               0
unnamed:_22               6301
unnamed:_23               6300
dtype: int64

In [69]:
#look for row's not in column's Unnamed: 22
df.loc[~df['unnamed:_22'].isnull(), :]

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
1478,2006.05.27,27-May-2006,2006.0,Unprovoked,USA,Hawaii,"North Shore, O'ahu",Surfing,Bret Desmond,M,...,NaN,R. Collier,2006.05.27-Desmond.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.05.27,2006.05.27,4825.0,stopped here,NaN


In [70]:
#look for row's not in column's Unnamed: 23
df.loc[~df['unnamed:_23'].isnull(), :]

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


In [71]:
#drop columns Unnamed
df.drop(labels=['unnamed:_22','unnamed:_23'], axis=1, inplace=True)

#### Work with 'Fatal (Y/N)' column

In [72]:
df['fatal_(y/n)'].unique()

array(['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y'],
      dtype=object)

In [73]:
#remove spaces before and after
df['fatal_(y/n)'] = df['fatal_(y/n)'].str.strip()
#lower all strings
df['fatal_(y/n)'] = df['fatal_(y/n)'].str.lower()
df['fatal_(y/n)'].unique()

array(['n', 'y', nan, 'm', 'unknown', '2017'], dtype=object)

In [74]:
#group other responses in missing value
df['fatal_(y/n)'] = df['fatal_(y/n)'].apply(lambda x: np.nan if x not in ['y','n'] else x)
df['fatal_(y/n)'].unique()

array(['n', 'y', nan], dtype=object)

#### Work with 'Sex' column

In [75]:
df['sex'].unique()

array(['F', 'M', nan, 'M ', 'lli', 'N', '.'], dtype=object)

In [76]:
#remove spaces before and after
df['sex'] = df['sex'].str.strip()
#lower all strings
df['sex'] = df['sex'].str.lower()
df['sex'].unique()

array(['f', 'm', nan, 'lli', 'n', '.'], dtype=object)

In [77]:
#group other responses in missing value
df['sex'] = df['sex'].apply(lambda x: np.nan if x not in ['f','m'] else x)
df['sex'].unique()

array(['f', 'm', nan], dtype=object)

## Answer the questions

In [113]:
# create data frame only with columns we want, with drop the other columns we don't need to this analyse
df_answer_1 = df.drop(['date','case_number','type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury','time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number.1', 'case_number.2', 'original_order'], axis =1)
# drop the lines with na
df_answer_1.dropna(inplace=True)
#filter year to 1900 +
df_answer_1 = df_answer_1.loc[df_answer_1['year'] >= 1900, :]
# create two columns fatals e non_fatals to sum after
df_answer_1['fatals'] = np.where(df_answer_1['fatal_(y/n)'] == 'y', True, False)
df_answer_1['non_fatals'] = np.where(df_answer_1['fatal_(y/n)'] == 'n', True, False)
#drop the column original
df_answer_1.drop(labels='fatal_(y/n)', axis=1, inplace = True)
#group by year and sum year
df_answer_1 = df_answer_1.groupby(by='year', as_index = False).sum()
#creating column total, sum of fatals and non_fatals
df_answer_1['Total'] = df_answer_1['fatals'] + df_answer_1['non_fatals'] 
#convert column year to int
df_answer_1['year'] =df_answer_1.astype(int)
df_answer_1

,year,fatals,non_fatals,Total
0,1900,3.0,7.0,10.0
1,1901,3.0,4.0,7.0
2,1902,5.0,8.0,13.0
3,1903,8.0,2.0,10.0
4,1904,10.0,3.0,13.0
...,...,...,...,...
114,2014,7.0,111.0,118.0
115,2015,9.0,121.0,130.0
116,2016,5.0,113.0,118.0
117,2017,8.0,115.0,123.0
